In [ ]:
from flask import Flask, request, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy
from flask_cors import CORS
import requests
from datetime import datetime
import pymysql

app = Flask(__name__, template_folder='templates')
CORS(app, resources={
    r"/api/*": {
        "origins": ["http://localhost:*", "http://127.0.0.1:*"],
        "methods": ["GET", "POST", "PUT", "DELETE"],
        "allow_headers": ["Content-Type"]
    }
})

# MySQL Configuration
app.config['SQLALCHEMY_DATABASE_URI'] = 'mysql+pymysql://root:@localhost/cozy_seller'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_POOL_RECYCLE'] = 299
app.config['SQLALCHEMY_POOL_SIZE'] = 20

db = SQLAlchemy(app)
DISTRIBUTOR_URL = 'http://localhost:5001/api'

# Database Models
class Product(db.Model):
    __tablename__ = 'products'
    id = db.Column(db.Integer, primary_key=True)
    blanket_model_id = db.Column(db.Integer, nullable=False, index=True)
    blanket_model_name = db.Column(db.String(100), nullable=False)
    description = db.Column(db.Text)
    selling_price = db.Column(db.Float, nullable=False)
    current_stock = db.Column(db.Integer, default=0)

    def to_dict(self):
        return {
            'id': self.id,
            'blanket_model_id': self.blanket_model_id,
            'blanket_model_name': self.blanket_model_name,
            'description': self.description,
            'selling_price': self.selling_price,
            'current_stock': self.current_stock
        }

class CustomerOrder(db.Model):
    __tablename__ = 'customer_orders'
    id = db.Column(db.Integer, primary_key=True)
    customer_name = db.Column(db.String(100), nullable=False)
    customer_email = db.Column(db.String(100), nullable=False)
    blanket_model_id = db.Column(db.Integer, nullable=False)
    blanket_model_name = db.Column(db.String(100), nullable=False)
    quantity = db.Column(db.Integer, nullable=False)
    total_price = db.Column(db.Float, nullable=False)
    status = db.Column(db.String(50), nullable=False, default='pending')
    order_date = db.Column(db.String(50), nullable=False)
    fulfillment_date = db.Column(db.String(50))

    def to_dict(self):
        return {
            'id': self.id,
            'customer_name': self.customer_name,
            'customer_email': self.customer_email,
            'blanket_model_id': self.blanket_model_id,
            'blanket_model_name': self.blanket_model_name,
            'quantity': self.quantity,
            'total_price': self.total_price,
            'status': self.status,
            'order_date': self.order_date,
            'fulfillment_date': self.fulfillment_date
        }

# Initialize Database
with app.app_context():
    db.create_all()

# Helper Functions
def validate_product_data(data):
    required_fields = ['blanket_model_id', 'blanket_model_name', 'selling_price']
    return all(field in data for field in required_fields)

def validate_order_data(data):
    required_fields = ['customer_name', 'customer_email', 'blanket_model_id', 'quantity']
    return all(field in data for field in required_fields)

# API Endpoints
@app.route('/')
def index():
    return render_template('seller.html')

@app.route('/api/products', methods=['GET'])
def get_products():
    try:
        products = Product.query.all()
        return jsonify([product.to_dict() for product in products])
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/products/<int:id>', methods=['GET'])
def get_product(id):
    try:
        product = Product.query.get_or_404(id)
        return jsonify(product.to_dict())
    except Exception as e:
        return jsonify({'error': str(e)}), 404

@app.route('/api/products', methods=['POST'])
def add_product():
    try:
        data = request.json
        if not validate_product_data(data):
            return jsonify({'error': 'Missing required fields'}), 400

        product = Product(
            blanket_model_id=data['blanket_model_id'],
            blanket_model_name=data['blanket_model_name'],
            description=data.get('description', ''),
            selling_price=data['selling_price'],
            current_stock=data.get('current_stock', 0)
        )
        db.session.add(product)
        db.session.commit()
        return jsonify({'message': 'Product added successfully', 'data': product.to_dict()}), 201
    except Exception as e:
        db.session.rollback()
        return jsonify({'error': str(e)}), 500

@app.route('/api/products/<int:id>', methods=['PUT'])
def update_product(id):
    try:
        product = Product.query.get_or_404(id)
        data = request.json
        
        if 'blanket_model_id' in data:
            product.blanket_model_id = data['blanket_model_id']
        if 'blanket_model_name' in data:
            product.blanket_model_name = data['blanket_model_name']
        if 'description' in data:
            product.description = data['description']
        if 'selling_price' in data:
            product.selling_price = data['selling_price']
        if 'current_stock' in data:
            product.current_stock = data['current_stock']
            
        db.session.commit()
        return jsonify({'message': 'Product updated successfully', 'data': product.to_dict()})
    except Exception as e:
        db.session.rollback()
        return jsonify({'error': str(e)}), 500

@app.route('/api/products/<int:id>', methods=['DELETE'])
def delete_product(id):
    try:
        product = Product.query.get_or_404(id)
        db.session.delete(product)
        db.session.commit()
        return jsonify({'message': 'Product deleted successfully'})
    except Exception as e:
        db.session.rollback()
        return jsonify({'error': str(e)}), 500

@app.route('/api/orders', methods=['GET'])
def get_orders():
    try:
        status_filter = request.args.get('status')
        query = CustomerOrder.query
        
        if status_filter:
            query = query.filter(CustomerOrder.status == status_filter)
            
        orders = query.order_by(CustomerOrder.order_date.desc()).all()
        return jsonify([order.to_dict() for order in orders])
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/orders', methods=['POST'])
def create_order():
    try:
        data = request.json
        if not validate_order_data(data):
            return jsonify({'error': 'Missing required fields'}), 400

        product = Product.query.filter_by(blanket_model_id=data['blanket_model_id']).first()
        selling_price = product.selling_price if product else data.get('selling_price', 0)
        
        # Try to fulfill from local stock
        if product and product.current_stock >= data['quantity']:
            product.current_stock -= data['quantity']
            order = CustomerOrder(
                customer_name=data['customer_name'],
                customer_email=data['customer_email'],
                blanket_model_id=data['blanket_model_id'],
                blanket_model_name=product.blanket_model_name,
                quantity=data['quantity'],
                total_price=selling_price * data['quantity'],
                status='fulfilled',
                order_date=data.get('order_date', datetime.now().strftime('%Y-%m-%d')),
                fulfillment_date=data.get('order_date', datetime.now().strftime('%Y-%m-%d'))
            )
            db.session.add(order)
            db.session.commit()
            return jsonify({'message': 'Order fulfilled from local stock', 'status': 'fulfilled'}), 201
        
        # Try to fulfill from distributor
        try:
            response = requests.post(
                f'{DISTRIBUTOR_URL}/orders',
                json={
                    'seller_id': 1,
                    'blanket_model_id': data['blanket_model_id'],
                    'quantity': data['quantity'],
                    'order_date': data.get('order_date', datetime.now().strftime('%Y-%m-%d'))
                },
                timeout=5
            )
            
            if response.status_code == 201:
                response_data = response.json()
                if 'fulfilled' in response_data['message'].lower():
                    order = CustomerOrder(
                        customer_name=data['customer_name'],
                        customer_email=data['customer_email'],
                        blanket_model_id=data['blanket_model_id'],
                        blanket_model_name=product.blanket_model_name if product else 'Unknown',
                        quantity=data['quantity'],
                        total_price=selling_price * data['quantity'],
                        status='fulfilled',
                        order_date=data.get('order_date', datetime.now().strftime('%Y-%m-%d')),
                        fulfillment_date=data.get('order_date', datetime.now().strftime('%Y-%m-%d'))
                    )
                    db.session.add(order)
                    db.session.commit()
                    return jsonify({'message': 'Order fulfilled from distributor', 'status': 'fulfilled'}), 201
                else:
                    return handle_backorder(data, product, selling_price)
            else:
                return handle_backorder(data, product, selling_price)
                
        except requests.exceptions.RequestException as e:
            return handle_failed_order(data, product, selling_price, str(e))
            
    except Exception as e:
        db.session.rollback()
        return jsonify({'error': str(e)}), 500

def handle_backorder(data, product, selling_price):
    order = CustomerOrder(
        customer_name=data['customer_name'],
        customer_email=data['customer_email'],
        blanket_model_id=data['blanket_model_id'],
        blanket_model_name=product.blanket_model_name if product else 'Unknown',
        quantity=data['quantity'],
        total_price=selling_price * data['quantity'],
        status='backordered',
        order_date=data.get('order_date', datetime.now().strftime('%Y-%m-%d')),
        fulfillment_date=None
    )
    db.session.add(order)
    db.session.commit()
    return jsonify({'message': 'Order backordered', 'status': 'backordered'}), 202

def handle_failed_order(data, product, selling_price, error_msg):
    order = CustomerOrder(
        customer_name=data['customer_name'],
        customer_email=data['customer_email'],
        blanket_model_id=data['blanket_model_id'],
        blanket_model_name=product.blanket_model_name if product else 'Unknown',
        quantity=data['quantity'],
        total_price=selling_price * data['quantity'],
        status='failed',
        order_date=data.get('order_date', datetime.now().strftime('%Y-%m-%d')),
        fulfillment_date=None
    )
    db.session.add(order)
    db.session.commit()
    return jsonify({'error': error_msg, 'status': 'failed'}), 500

if __name__ == '__main__':
    app.run(port=5002, debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5002
Press CTRL+C to quit
127.0.0.1 - - [04/Aug/2025 01:46:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2025 01:46:21] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Aug/2025 01:46:21] "GET /api/products HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2025 01:46:21] "GET /api/orders HTTP/1.1" 200 -
